<a href="https://colab.research.google.com/github/Lutris98/API_Cheatsheets/blob/main/Sklearn_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pandas ver1.2.1<br>
Sklearn ver0.24.1

#1.EDA

■ Pandas(with Numpy) 

In [ ]:
import pandas as pd

•불러오기

In [ ]:
titanic_df=pd.read_csv(’C:\\Users\\john\\titanic_train.csv’) #csv는 ,로 구별한 파일양식 
from google.colab import drive
drive.mount('content/gdrive')
titanic_df=pd.read_csv('/content/gdrive/Mydrive/Dataset/titanic_train.csv')

•만들기

In [ ]:
df=pd.DataFrame(ndarray,columns=['col1','col2']) 
ndarray=df.values #ndarray가 필요한 작업들이 있다(특히 columns series를 for루프에 넣을 리스트로 만들때)

•Numpy(list약간확장)

In [ ]:
ndarray=np.array([1,2,3]) #list랑 약간 다름 #np.arange(10), np.zeros((3,2))로 편하게 생성가능
array2=array1.reshape(2,3) #차원변경 #-1넣으면 호환되는 값을 대신 대입

In [ ]:
sort_indices=np.argsort(score_array) 
print(name_array[sort_indices]) #메타데이터가 없는 ndarray에서 활용도 높음

In [ ]:
np.dot(A,B) 
np.transpose(A)

##1)Basic statistics

In [ ]:
titanic_df.head(3) #상위 세건 직접 보기
titanic_df.info() #dtype,Null개수,outlier
titanic_df.describe() #기초통계량

##2)Basic feature selection(for preprocessing)

In [ ]:
titanic_df.drop(['Age','Sex'],axis=1,inplace=True) #axis필수,웬만하면 inplace

In [ ]:
titanic_df['New_feature']=[1,2,3] #칼럼 추가
titanic_df['Family']=titianic_df['SibSp']*10+titanic_df['Parch']
titanic_df['Name_len']=titanic_df['Name'].apply(lambda:len(x)) #len(titanic['Name])이 아니라 apply lambda

•불린 인덱싱

In [ ]:
titanic_boolean=titanic_df[(titanic_df['Age']>60)&(titanic_df['Sex']=='female')] #새로운 dataframe만들때
df.loc[(df.Age>15)&(df.Age<=30), 'Age_band']=2 #새로운 feature만들때

##3)Encoding

In [ ]:
df=pd.get_dummies(df,columns=['workingday','hour','weather','month']) #onehotencoder쉬운버전(encoder들과 다르게 df가 input)

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder #categorical에서 numerical로/categorical값마다 새 칼럼만들기
items=['TV','냉장고','전자레인지','TV'] 
le=LabelEncoder().fit_transform(items)
le=le.reshape(-1,1) #need 2darray
ohe=OneHotEncoder().fit_transform(le) #input으로 list가능/불가능이라 ohe는 le의 output을 넣는다

#2.Preprocessing(with Visualization)

•시각화

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
fig,ax=plt.subplots(figsize=(16,8),ncols=4,nrows=2) #figsize로 figure객체의 픽셀조절 #subplot은 위치값인 axs도
features=['RM','ZN','INDUS']
for i,feature in enumerate(features):
    row=int(i/4) #각 figure배치는 직접 ax보고 확인
    col=i%4
    ax=sns.histplot(data=DF,X=feature,y='PRICE',ax=axs[row][col]) #간단한 회귀계산에 그래프까지
    ax=sns.bar

##1)Null filling

In [ ]:
titanic_df['Cabin']=titanic_df['Cabin'].fillna(0)
titanic_df['Cabin'].isna().sum() #확인

##2)Scaling&Outlier removal(Only regression)

##3)Feature selection

•Filter방식<br>
target과 각 feature의 통계학적 연관성<br>
chi2:categorical feature(mutual_info_classif도 가능)<br>
수학적으로 category값별 target값의 실제값,예측값 비교<br>
f_classif:numerical feature<br>
수학적으로 target별 numerical값의 평균이 다름(ANOVA방식)

In [ ]:
from sklearn.feature_selection import SelectKBest,chi2,mutual_info_classif,f_classif
X=SelectKBest(chi2,k=2).fit_transform(X,y) #SelectPercentile(chi2,percentile=10)도 괜찮

In [ ]:
df.corr.where(np.triu(np.ones(df.corr)).astype(np.bool_),inplace=True) #upper corr로 만들기(True,1이면 남김) #triangular upper #bool means trueorfalse
to_drop=[column for column in upper.columns if any(upper[column]>0.9)] #[return값 for문 똑같음 if조건문] #corr matrix를 column단위로 분석
df.drop(to_drop,inplace=True)

•Greedy방식<br>
-Recursive<br>
-Forward/Backward<br>

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
sfs=SFS(RandomForestRegressor(),forward=False,scoring='neg_mean_squared_error',k_features=10,cv=3)

•Embedded방식<br>

In [ ]:
sel=SelectFromModel(Lasso(alpha=100))
sel.fit(X,y)
X=X[sel.get_support()] #get_support는 객체반환(직접해봐야알듯)

In [ ]:
importances=pd.Series(rf_clf.feature_importances_,index=X.columns) #자체로는 ndarray
importances.sort_values()
sns.barplot(x=importances,y=importances.index)

##4)Dimension reduction(Too many features)

•PCA<br>
공분산행렬(symmetric)을 EVD, 고유값 첫번째, 두번째큰 고유벡터로 분해 

In [ ]:
from sklearn.decomposition import PCA 
pca=PCA(n_components=6)
santander_pca=pca.fit_transform(santander_scaled) #보통 scale하고 적용 #ndarray반환
pca_columns=['pca1','pca2','pca3','pca4','pca5','pca6']
santander_df=pd.DataFrame(santander_pca,columns=pca_columns)

•Truncated SVD<br> 
PCA와 다르게 rectangular로 확장<br>
∑최소로 diagonal하게 만들고 U,Vt도 맞춰주기

•LDA<br>
클래스간 분산최대, 클래스내 분산최소

In [ ]:
from sklearn.dicriminant_analysis import LinearDiscriminantAnalysis
lda=LinearDiscriminantAnalysis(n_components=6)
lda.fit_transform(santander_scaled,santander['target']) #target값까지 필요

#3.Modeling

In [ ]:
from sklearn.model_selection import train_test_split,GridSearchCV, KFold, StratififedKFold #Stratified는 label분포가 치중되었을때(주로 분류)

In [ ]:
X_train,X_test,y_train,y_test= train_test_split(titanic_feature,titanic_label,test_size=0.2,random_state=0) #X가 먼저 #재현을 위해 randomstate필수

In [ ]:
kfold=KFold(n_splits=5,random_state=0) #5개의 kfold객체 생성 
for train_index,test_index in kfold.split(titanic_feature,titanic_label): #각 인덱스를 ndarray로 반환 
  X_train,X_test=titanic_feature[train_index], titanic_feature[test_index]
  y_train,y_test=titanic_label[train_index], titanic_label[test_index]
  lgbm_clf.fit(X_train,y_train)
  cv_accuracy.append(accuracy_score(y_test,lgbm.predict(X_test)))
  print('cv정확도:',cv_accuracy)
print('\n평균cv정확도:',np.mean(cv_accuracy))

■ 분류

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

• RandomForest(Bagging)<br>
병렬적으로 결정트리에 bootstrapping된 데이터 학습<br>
DecisionTree기반 알고리즘은 회귀도 가능(exRandomForestRegressor)

In [ ]:
rf_clf=RandomForestClassifier(random_state=0)
params={'n_estimators':[100] #100개의 tree에서 나온 확률값 평균
        'max_depth':[6,8,10] #이하 3개는 결정트리것 그대로
        'min_samples_leaf':[8,12] #samples수 너무 적을 때까지 않가도록(클수록 과적합방지)
        'min_samples_split':[8,16] #split기준은 지니계수(한쪽이 우세한 정도라 직관적)
        }
rf_gridcv.fit=GridSearchCV(rf_clf,param_grid=params,cv=5)
rf_gridcv.fit(X_train,y_train)

•XGBoost(Boosting)<br> 
GBM대비 병렬학습가능해 빠르고 과적합규제/가지치기로 split재검토,내부적 교차검증,결손값처리,조기중단

In [ ]:
xgb_clf=XGBClassifier(random_state=0)
params={'learning_rate':[0.02],
        'n_estimators':[1000,1500],
        'max_depth':[3,5,7],
        'colsample_bytree':[0.7,0.8],
        'subsample':[0.9]}

•LightGBM(Boosting)<br> 
XGBoost와 달리 leafwise 분할(시간,공간복잡도 개선, 성능비슷)

In [ ]:
lgbm_clf=LGBMClassifier(random_state=0)
params={'learning_rate':[0.02],
        'n_estimators':[1000,1500],
        'max_depth':[3,5,7], #leafwise니까 살짝 더 크게
        'colsample_bytree':[0.7,0.8], 
        'subsample':[0.9]
        }
grid_cv.fit=GridSearchCV(lgbm_clf,param_grid=params,cv=5)
grid_cv.fit(X_train,y_train,early_stopping_rounds=30,eval_metric='auc',eval_set=[(X_train,y_train),(X_test,y_test)] #향상되지 않고 반복할 수 있는 최대횟수 #eval_set은 보통 학습데이터셋말고 다른것 

•LogisticRegression<br>
sigmoid함수반환값을 확률로 간주<br>
penalty로 규제유형(디폴트는 l2) / C로 alpha역수

•SVC<br>
feature space에서 가장 넓은 도로를 놓을 수 있는 선(large marginal classifictation)<br>
도로안에 가장 많은 sample들어가게 하는 선으로 회귀도 가능(파라미터는 epsilon=1.5)<br>
kernel trick방식은 속도가 좀 느림(유사도나 다항처리나)
비교적 작은 데이터셋에 적합

In [ ]:
from sklearn.svm import LinearSVC,SVC
svm_clf=LinearSVC(C=1,loss='hinge') #C는 violation 낮을수록 설명력 향상

In [ ]:
svm_clf=SVC(kernel='rbf',gamma=5,c=0.001) #gamma는 높을수록 설명력 향상(각 landmark의 영향력) #landmark와의 유사도(RadialBasisFunction)

•Metric<br>
confusion matrix에서 FP와 FN줄이기

In [ ]:
from sklearn.metrics import recall_score,precision_score,f1_score,roc_auc_score
def get_clf_eval(real,pred):
  recall=recall_score(real,pred) #FP줄이고 싶음(암진단모델)
  precision=precision_score(real,pred) #FN줄이고 싶음(스팸차단모델)
  f1=f1_score(real,pred) #치우치지 않은 정도
  auc=roc_auc_score(real,pred) #TPR-FPR그래프에서 TPR이 얼마나 큰지(1에 가까울수록 우수)
  print('recall score:{0:.4f} / precision score:{1:.4f} / f1 score:{2:.4f} / f1 score:{3:.4f}'.format(recall,precision,f1,auc)) 

■ 회귀

•Scaling<br>
Feature,Target을 로그변환

In [ ]:
def get_scaled_data(method=None,data=None): #적어도 target은 무조건 scaling 필요
    if method='Standard':scaled_data=StandardScaler().fit_transform(data)
    elif method='MinMax':scaled_data=MinMaxScaler().fit_transform(data)
    elif method='Log':scaled_data=np.log1p(data) #변경된 target값으로 학습후 결과를 다시 expm1로 되돌림 #log(1+x)
    else:scaled_data=data
    return scaled_data

•PolynomialFeatures<br>
주어진 변수로 파생변수추가해 설명력향상 but overfit 위험, 변수 너무 많아짐(커봤자 degree3)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
feature_df=PolynomialFeatures(degree=3).fit_transform(feature_df) 

•Regulation<br>

In [ ]:
from sklearn.linear_model import Ridge,Lasso,ElasticNet
Ridge(alpha=0.1).fit(X_train,y_train) #커질수록 설명력 감소 #L2:계수크기규제(RSS+alpha*W2최소화)
Lasso(alpha=0.1).fit(X_train,y_train) #L1:계수종류규제->feature selection기여(RSS+alpha*W최소화)
ElasticNet(alpha=0.1,l1_ratio=0.7).fit(X_train,y_train) #alpha는 합 

•Metric<br>
MSE, R2

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score #MSE와 R2
def get_clf_eval(real,pred):
  mse=mean_squared_error(real,pred)
  r2=r2_score(real,pred)
  print('MSE:{0:.4f} / R2:{1:.4f}'.format(mse,r2)) 

■ 군집화

•Kmeans<br>
centroid별 평균중심으로 업데이트

In [ ]:
from sklearn.cluster import KMeans
kmeans_clus=KMeans(n_clusters=3,max_iter=300)
titanic_df['cluster']=kmeans_clus.fit(test_df)
titanic_df.groupby(['target','cluster'])['Cabin'].count 

•MeansShift<br>
centroid별 밀도중심으로 업데이트(KernelDensityFunction)

In [ ]:
from sklearn.cluster import MeanShift, estimate_bandwidth
meanshift_clus=MeanShift(bandwidth=estimate_bandwidth) #최적의 bandwidth값 return(작아야 설명력 향상)

•DBSCAN<br>

In [ ]:
from sklearn.cluster import DBSCAN #min_samples 넘는 corepoint끼리 연결/적으면 borderpoint거나 noise
dbscan=DBSCAN(eps=0.6,min_samples=8,metric='euclidean') #eps클수록 border많아짐/min_samples클수록 core적어짐

•Metric<br>
silhouette coef<br>
각 데이터마다 b-a/max(b,a) / a는 클러스터내 평균거리, b는 최근접 클러스터내 평균거리 


In [ ]:
from sklearn.metrics import silhouette_samples
scores=silhouette_samples(titanic_df,titanic_df['cluster']) #모든 데이터의 score저장(1일수록 우수)